In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 80)
idx = pd.IndexSlice

## Parser Excel fil og lagert tabeller for seire og plassering

In [2]:
new_col_names = ['Division','Team','Place in division','Regular season wins']
xl = pd.ExcelFile('NFL tippekonk input 2019.xlsx')

df_NFL = []
for i,sheet_name in enumerate(xl.sheet_names):

    df_AFC = pd.read_excel('NFL tippekonk input 2019.xlsx',sheet_name=sheet_name,nrows=19,header=3,usecols=[0,1,2,3])
    df_AFC = df_AFC.rename(columns=dict(zip(df_AFC.columns.values,new_col_names)))
    df_AFC = df_AFC.dropna(how='all')
    df_AFC.insert(0,'Conference','AFC')
    df_AFC = df_AFC.fillna(method='ffill')


    df_NFC = pd.read_excel('NFL tippekonk input 2019.xlsx',sheet_name=sheet_name,nrows=19,header=3,usecols=[5,6,7,8])
    df_NFC = df_NFC.rename(columns=dict(zip(df_NFC.columns.values,new_col_names)))
    df_NFC = df_NFC.dropna(how='all')
    df_NFC.insert(0,'Conference','NFC')
    df_NFC = df_NFC.fillna(method='ffill')
    df_NFC

    df_NFL.append(pd.concat([df_AFC,df_NFC],axis=0,ignore_index=True))
    df_NFL[i].insert(0,'Name',sheet_name)


df_NFL = pd.concat(df_NFL,axis=0,ignore_index=True)
for i in [4,5]:
    df_NFL.iloc[:,i] = pd.to_numeric(df_NFL.iloc[:,i],downcast='unsigned')

In [3]:
placement = df_NFL.pivot_table(values=['Place in division'],columns='Name',index=['Conference','Division','Team'])
placement.columns = placement.columns.droplevel()
wins = df_NFL.pivot_table(values=['Regular season wins'],columns='Name',index=['Conference','Division','Team'])
wins.columns = wins.columns.droplevel()

In [4]:
max_guessers = wins.apply(lambda x: ', '.join(x[x.values == x.values.max()].index.values),axis=1)
min_guessers = wins.apply(lambda x: ', '.join(x[x.values == x.values.min()].index.values),axis=1)
seiersforskjell = pd.DataFrame([max_guessers,wins.max(axis=1),min_guessers,wins.min(axis=1),(wins.max(axis=1)-wins.min(axis=1))]).T
seiersforskjell.columns=['Highest guesser','High guess','Lowest guesser','Low guess','Difference']
seiersforskjell = seiersforskjell.sort_values(by='Difference',ascending=False).reset_index(drop=False)
seiersforskjell

,Conference,Division,Team,Highest guesser,High guess,Lowest guesser,Low guess,Difference
0,NFC,East,Eagles,Kjetil,12,Anders,4,8
1,AFC,East,Dolphins,Anders,9,David,1,8
2,AFC,West,Broncos,Anders,10,Kjetil,3,7
3,AFC,South,Titans,"Eirik, Yngve",8,Kjetil,1,7
4,AFC,East,Jets,David,9,Anders,2,7
5,AFC,North,Bengals,Anders,10,Kjetil,3,7
6,AFC,North,Browns,"Andreas, Yngve",10,Anders,3,7
7,AFC,West,Raiders,Kjetil,8,Anders,1,7
8,NFC,North,Lions,Anders,10,David,4,6
9,NFC,North,Bears,"Andreas, David, Yngve",11,Anders,5,6


In [5]:
max_guessers = placement.apply(lambda x: ', '.join(x[x.values == x.values.max()].index.values),axis=1)
min_guessers = placement.apply(lambda x: ', '.join(x[x.values == x.values.min()].index.values),axis=1)
plasseringsforskjell = pd.DataFrame([min_guessers,placement.min(axis=1),max_guessers,placement.max(axis=1),(placement.max(axis=1)-placement.min(axis=1))]).T
plasseringsforskjell.columns=['Most optimistic guesser','Most optimistic guess','Most pessimistic guesser','Most pessimistic guess','Difference']
plasseringsforskjell = plasseringsforskjell.sort_values(by='Difference',ascending=False).reset_index(drop=False)
plasseringsforskjell

,Conference,Division,Team,Most optimistic guesser,Most optimistic guess,Most pessimistic guesser,Most pessimistic guess,Difference
0,NFC,North,Bears,"David, Kjetil, Kristian, Yngve",1,Anders,4,3
1,AFC,North,Bengals,Anders,1,"Andreas, David, Eirik, Kjetil, Kristian, Markus, Yngve",4,3
2,AFC,North,Browns,"Andreas, Kristian",1,Anders,4,3
3,NFC,East,Eagles,"Eirik, Kjetil, Kristian, Markus, Yngve",1,Anders,4,3
4,AFC,South,Colts,Yngve,1,Eirik,4,3
5,NFC,North,Vikings,Andreas,1,Eirik,4,3
6,AFC,West,Broncos,"Anders, David",2,"Eirik, Kjetil, Kristian, Markus, Yngve",4,2
7,NFC,East,Redskins,Anders,2,"Andreas, Eirik, Kjetil, Kristian, Markus, Yngve",4,2
8,NFC,North,Packers,"Anders, Eirik, Markus",1,"Andreas, Kjetil",3,2
9,AFC,East,Dolphins,Anders,2,"Andreas, David, Eirik, Kjetil, Yngve",4,2


In [6]:
wins.sum(axis=0).sort_values(ascending=False)

Name
Yngve       256
Eirik       256
Andreas     256
Anders      247
Kjetil      245
David       241
Markus      236
Kristian    222
dtype: int64

In [7]:
for i,sheet_name in enumerate(xl.sheet_names):

    wins_sum = placement[sheet_name].groupby(by=['Conference','Division']).sum()
    n = wins_sum[(placement[sheet_name].groupby(by=['Conference','Division']).nunique() != 4) | (wins_sum != 10)].shape[0]
    print('For {0:s} we have {1:d} cases of wrong ranking numbers for a division'.format(sheet_name,n))
    
    
    df = pd.DataFrame([placement[sheet_name],wins[sheet_name]],index=['Placement','Wins']).T
    df = df.sort_values(by=['Conference','Division','Placement'])
    bol_1 = (df['Wins']-df['Wins'].shift(-1) < 0) & (df['Placement'] != 4)
    bol_2 = bol_1.shift(1).fillna(False)
    mismatches = df.loc[bol_1 | bol_2,:]
    
    n = mismatches.shape[0]//2
    print('For {0:s} we have the following {1:d} mismatch(es) between rankings and wins'.format(sheet_name,n))
    if n > 0:
        print(mismatches)
    print('\n')

    temp = df['Placement'].values 
    df = df.sort_values(by=['Conference','Division','Wins','Placement'],ascending=[True,True,False,True])
    df['Placement'] = temp
    placement[sheet_name] = df['Placement']

For Anders we have 0 cases of wrong ranking numbers for a division
For Anders we have the following 1 mismatch(es) between rankings and wins
                             Placement  Wins
Conference Division Team                    
AFC        South    Titans           3     3
                    Jaguars          4     8


For Andreas we have 0 cases of wrong ranking numbers for a division
For Andreas we have the following 0 mismatch(es) between rankings and wins


For David we have 0 cases of wrong ranking numbers for a division
For David we have the following 0 mismatch(es) between rankings and wins


For Eirik we have 0 cases of wrong ranking numbers for a division
For Eirik we have the following 1 mismatch(es) between rankings and wins
                             Placement  Wins
Conference Division Team                    
AFC        West     Raiders          3     6
                    Broncos          4     7


For Kjetil we have 0 cases of wrong ranking numbers for a division
For

In [8]:
min_rankings = ((placement==1)*17+wins).groupby(by='Conference').rank(method='min',ascending=False).astype('int')
max_rankings = ((placement==1)*17+wins).groupby(by='Conference').rank(method='max',ascending=False).astype('int')
for i,sheet_name in enumerate(xl.sheet_names):
    playoffs = pd.read_excel('NFL tippekonk input 2019.xlsx',sheet_name=sheet_name,nrows=27,header=29,usecols=[0,2,3])
    playoffs = playoffs.dropna(how='all').reset_index(drop=True)
    playoffs.iloc[:,0] = playoffs.iloc[:,0].fillna(method='ffill')
    for col in ['AFC','NFC']:
        playoffs.loc[:,col] = playoffs.loc[:,col].str.title()
        if col == 'NFC':
            playoffs.loc[:,col] = playoffs.loc[:,col].replace('49Ers','49ers')

    rankings = pd.concat([min_rankings[sheet_name],max_rankings[sheet_name]],axis=1)
    rankings.columns=['Low rank','High rank']
    rankings = rankings.sort_values(by=['Conference','Low rank'])
    playoff_teams = rankings[rankings['Low rank'] < 7].droplevel(level=1).reset_index()
    
    for conference in ['AFC','NFC']:
        print('*'*20 + sheet_name + '-' + conference + '*'*20)
        target_playoff = playoffs.loc[0:5,conference]
        mismatches = target_playoff[~target_playoff.isin(playoff_teams.loc[playoff_teams['Conference']==conference,'Team'])].values
        print('For the {0:s}, {1:s} has {2:d} mismatched playoff teams: {3:s}'.format(conference,sheet_name,len(mismatches),', '.join(mismatches)))
        
        target_bye = playoffs.loc[6:7,conference]
        universe = playoff_teams.loc[(playoff_teams['Conference']==conference) & (playoff_teams['Low rank'] < 3),'Team']
        mismatches = target_bye[~target_bye.isin(universe)].values
        print('For the {0:s}, {1:s} has {2:d} mismatched bye teams: {3:s}'.format(conference,sheet_name,len(mismatches),', '.join(mismatches)))
    
        target_div = playoffs.loc[8:9,conference]
        universe = playoff_teams.loc[(playoff_teams['Conference']==conference) & (playoff_teams['Low rank'] < 5) & (playoff_teams['High rank'] > 2),'Team']
        mismatches = target_div[~target_div.isin(universe)].values
        print('For the {0:s}, {1:s} has {2:d} mismatched 3-4 seed teams: {3:s}'.format(conference,sheet_name,len(mismatches),', '.join(mismatches)))
        
        target_wild = playoffs.loc[10:11,conference]
        universe = playoff_teams.loc[(playoff_teams['Conference']==conference) & (playoff_teams['High rank'] > 4),'Team']
        mismatches = target_wild[~target_wild.isin(universe)].values
        print('For the {0:s}, {1:s} has {2:d} mismatched wild card teams: {3:s}'.format(conference,sheet_name,len(mismatches),', '.join(mismatches)))
        
        
        rankings.loc[idx[:,:,target_bye.values],:] = rankings.loc[idx[:,:,target_bye.values],:]-0.6
        rankings.loc[idx[:,:,target_div.values],:] = rankings.loc[idx[:,:,target_div.values],:]-0.4
        rankings.loc[idx[:,:,target_wild.values],:] = rankings.loc[idx[:,:,target_wild.values],:]-0.2
        rankings.loc[conference,'Low rank'] = rankings.loc[conference,'Low rank'].rank(method='min').values
        rankings.loc[conference,'High rank'] = rankings.loc[conference,'High rank'].rank(method='max').values 
       
        rankings = rankings.astype('int')
        print(rankings.loc[conference,:].sort_values(by=['Low rank']))
        n = (rankings.loc[idx[:,:,playoffs.loc[12:15,conference]],'Low rank'] <= 2).sum()
        
        print('For the {0:s}, {1:s} has {2:d} bye week teams in the divisional round'.format(conference,sheet_name,n))
        print(rankings.loc[idx[:,:,playoffs.loc[12:15,conference]],'Low rank'].nsmallest(2))
        print('\n')
        
        divisional_n_low = rankings.loc[idx[:,:,playoffs.loc[12:15,conference]],'Low rank'].nlargest(2).sum()
        divisional_n_high = rankings.loc[idx[:,:,playoffs.loc[12:15,conference]],'High rank'].nlargest(2).sum()
        
        if (divisional_n_low != 9) | (divisional_n_high != 9):
            print('For the {0:s}, {1:s} has correct winners from wild card weekend'.format(conference,sheet_name,n))
        else:
            print('For the {0:s}, {1:s} has two winners for wild card weekend that would have played each other'.format(conference,sheet_name,n))
            print(rankings.loc[idx[:,:,playoffs.loc[12:15,conference]],'Low rank'].nlargest(2))
        print('\n')
        
        div_rank = rankings.loc[idx[:,:,playoffs.loc[12:15,conference]],'Low rank'].rank(method='min').astype(int).droplevel([0,1])
        conference_n = div_rank.loc[playoffs.loc[16:17,conference]].sum()
        size_conf = div_rank.loc[playoffs.loc[16:17,conference]].shape[0]
        
        if (conference_n != 5) & (size_conf==2):
            print('For the {0:s}, {1:s} has correct winners from divisonal weekend'.format(conference,sheet_name,n))
        else:
            print('For the {0:s}, {1:s} has two winners for divisional weekend that would have played each other'.format(conference,sheet_name,n))
            print(div_rank.loc[playoffs.loc[16:17,conference]])
            
        print('\n')
        
    print('\n')
                
        

********************Anders-AFC********************
For the AFC, Anders has 0 mismatched playoff teams: 
For the AFC, Anders has 0 mismatched bye teams: 
For the AFC, Anders has 0 mismatched 3-4 seed teams: 
For the AFC, Anders has 0 mismatched wild card teams: 
                   Low rank  High rank
Division Team                         
West     Chiefs           1          1
East     Patriots         2          2
South    Texans           3          3
North    Bengals          4          4
West     Broncos          5          5
East     Dolphins         6          6
North    Steelers         7          8
South    Colts            7          8
         Jaguars          9          9
North    Ravens          10         11
West     Chargers        10         11
East     Bills           12         12
North    Browns          13         14
South    Titans          13         14
East     Jets            15         15
West     Raiders         16         16
For the AFC, Anders has 2 bye week t

C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib

Conference  Division  Team  
NFC         South     Saints    1
            West      Rams      2
Name: Low rank, dtype: int32


For the NFC, Andreas has correct winners from wild card weekend


For the NFC, Andreas has correct winners from divisonal weekend




********************David-AFC********************
For the AFC, David has 0 mismatched playoff teams: 
For the AFC, David has 0 mismatched bye teams: 
For the AFC, David has 0 mismatched 3-4 seed teams: 
For the AFC, David has 0 mismatched wild card teams: 
                   Low rank  High rank
Division Team                         
West     Chiefs           1          1
East     Patriots         2          2
North    Steelers         3          4
South    Texans           3          4
North    Browns           5          6
West     Broncos          5          6
East     Jets             7          7
South    Colts            8          8
East     Bills            9         11
South    Jaguars          9         11
West     Char

C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib

                   Low rank  High rank
Division Team                         
East     Patriots         1          1
West     Chiefs           2          2
North    Steelers         3          4
South    Texans           3          4
West     Chargers         5          5
North    Browns           6          6
East     Jets             7         10
North    Ravens           7         10
South    Jaguars          7         10
         Titans           7         10
East     Bills           11         13
South    Colts           11         13
West     Broncos         11         13
North    Bengals         14         15
West     Raiders         14         15
East     Dolphins        16         16
For the AFC, Eirik has 2 bye week teams in the divisional round
Conference  Division  Team    
AFC         East      Patriots    1
            West      Chiefs      2
Name: Low rank, dtype: int32


For the AFC, Eirik has correct winners from wild card weekend


For the AFC, Eirik has correct winne

C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib

********************Kristian-AFC********************
For the AFC, Kristian has 0 mismatched playoff teams: 
For the AFC, Kristian has 0 mismatched bye teams: 
For the AFC, Kristian has 0 mismatched 3-4 seed teams: 
For the AFC, Kristian has 0 mismatched wild card teams: 
                   Low rank  High rank
Division Team                         
East     Patriots         1          1
West     Chiefs           2          2
North    Steelers         3          4
South    Texans           3          4
West     Chargers         5          5
North    Browns           6          6
East     Jets             7          9
North    Ravens           7          9
West     Raiders          7          9
East     Dolphins        10         11
South    Colts           10         11
East     Bills           12         15
North    Bengals         12         15
South    Jaguars         12         15
West     Broncos         12         15
South    Titans          16         16
For the AFC, Kristian has 

C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\Users\Eirik\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: PerformanceWarning: indexing past lexsort depth may impact performance.
C:\Users\Eirik\Anaconda3\lib

                     Low rank  High rank
Division Team                           
North    Packers            1          1
West     Rams               2          2
South    Saints             3          3
East     Eagles             4          4
West     Seahawks           5          5
North    Vikings            6          6
South    Falcons            7          7
East     Cowboys            8          8
North    Bears              9          9
South    Panthers          10         11
West     49ers             10         11
North    Lions             12         12
East     Giants            13         15
South    Buccaneers        13         15
West     Cardinals         13         15
East     Redskins          16         16
For the NFC, Markus has 1 bye week teams in the divisional round
Conference  Division  Team   
NFC         North     Packers    1
            South     Saints     3
Name: Low rank, dtype: int32


For the NFC, Markus has correct winners from wild card weekend


F